In [1]:
import numpy as np
import tensorflow as tf
import cv2
from ultralytics import YOLO
from norfair import Detection, Tracker
import uuid


In [12]:
video_path = "../data/people_walking.mp4" #video path

 #"n" so it is nano model, faster. 
 #It is trained by COCO dataset that includes 80 different labels 
 #I used nano model but you can use small, medium,...
model = YOLO("../yolov8n.pt")

In [13]:
interpreter = tf.lite.Interpreter(model_path="../reid_model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [14]:
def get_embedding(image):
    image = cv2.resize(image, (128,256))
    image = image / 255.0
    image = np.expand_dims(image.astype(np.float32), axis=0)
    interpreter.set_tensor((input_details[0]['index']), image)
    interpreter.invoke()
    return interpreter.get_tensor(output_details[0]['index'])[0]

In [15]:
def cosine_similarity(a, b):
    return np.dot(a, b) /(np.linalg.norm(a) * np.linalg.norm(b))

In [16]:
tracker = Tracker(distance_function="euclidean", distance_threshold=25)